In [ ]:
pip install numpy  scikit-learn tensorflow transformers fasttext matplotlib seaborn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313462 sha256=74ff69a1fe57c6370f0fcd3aa3bd03a5b361f336a3ab182ce166be676dd6194e
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
df = pd.read_csv("/content/1429_1.csv")  # Update with correct file path
X = df['reviews.text'].astype(str)  # Ensure reviews are strings
y = df['reviews.rating']

<ipython-input-3-9f63c1f5530c>:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/1429_1.csv")  # Update with correct file path


In [ ]:
y_binary = np.where(y <= 2, 0, 1)

In [ ]:
positive_samples = np.sum(y_binary == 1)
negative_samples = np.sum(y_binary == 0)

if positive_samples > negative_samples:
    positive_indices = np.where(y_binary == 1)[0]
    negative_indices = np.where(y_binary == 0)[0]

    np.random.shuffle(positive_indices)
    positive_indices = positive_indices[:negative_samples]  # Balance classes

    balanced_indices = np.concatenate([positive_indices, negative_indices])
    np.random.shuffle(balanced_indices)

    X = X.iloc[balanced_indices]
    y_binary = y_binary[balanced_indices]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
# **Voting Classifier**
ensemble = VotingClassifier(
    estimators=[
        ('mlp', MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=300, activation='relu', solver='adam')),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('svm', SVC(kernel='linear', probability=True)),
        ('nb', MultinomialNB())  # Naive Bayes works well with TF-IDF
    ],
    voting='soft'  # Soft voting for better probability handling
)

In [ ]:
# **Train the Model**
ensemble.fit(X_train_tfidf, y_train)

VotingClassifier(estimators=[('mlp',
                              MLPClassifier(hidden_layer_sizes=(128, 64),
                                            max_iter=300)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svm', SVC(kernel='linear', probability=True)),
                             ('nb', MultinomialNB())],
                 voting='soft')

In [ ]:
y_pred_ensemble = ensemble.predict(X_test_tfidf)


In [ ]:
accuracy = accuracy_score(y_test, y_pred_ensemble)
precision = precision_score(y_test, y_pred_ensemble)
recall = recall_score(y_test, y_pred_ensemble)
f1 = f1_score(y_test, y_pred_ensemble)
conf_matrix = confusion_matrix(y_test, y_pred_ensemble)

print(f"\nEnsemble Classifier Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:\n", conf_matrix)


Ensemble Classifier Accuracy: 0.8769
Precision: 0.8903
Recall: 0.8571
F1 Score: 0.8734

Confusion Matrix:
 [[147  17]
 [ 23 138]]


In [ ]:
while True:
    user_input = input("\nEnter a review (or type 'exit' to quit): ").strip()

    if user_input.lower() == 'exit':
        print("👋 Exiting...")
        break

    if not user_input:
        print("⚠️ Empty input! Please enter a valid review.")
        continue

    user_input_tfidf = vectorizer.transform([user_input])
    y_pred_user = ensemble.predict(user_input_tfidf)

    sentiment_label = "Positive" if y_pred_user == 1 else "Negative"
    print(f"\nPredicted Sentiment: {sentiment_label}")



Enter a review (or type 'exit' to quit): this product is superb

Predicted Sentiment: Positive

Enter a review (or type 'exit' to quit): exit
👋 Exiting...
